In [1]:
import pvdeg

import pandas as pd
import xarray as xr

from dask.distributed import LocalCluster, Client
from dask_jobqueue import SLURMCluster

In [ ]:
cluster = SLURMCluster(
    queue="shared",
    account="inspire",
    cores=6,
    memory="80 GB",
    processes=True,
    log_directory="/home/tford/dev/dask-logs",
    walltime="02:00:00",  # Request a longer time
)
cluster.scale(32)

client = Client(cluster)

print(client.dashboard_link)

In [2]:
workers = 8

cluster = LocalCluster(
    n_workers=workers,
    processes=6,
)

client = Client(cluster)

print(client.dashboard_link)

/home/tford/.conda-envs/rpp/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38159 instead
  warnings.warn(


http://127.0.0.1:38159/status


In [ ]:
locationGetter = pvdeg.scenario.GeospatialScenario()

locationGetter.addLocation(country="United States", downsample_factor=9)

In [ ]:
locationGetter.plot_coords()

In [ ]:
# this shows us that the data for poa_front is repeated 25 times for tmy, all years are the samer
res = pvdeg.pysam.pysam(
    pvdeg.weather.roll_tmy(
        locationGetter.weather_data.isel(gid=0).to_dataframe(),
        locationGetter.meta_data.iloc[0].to_dict(),
    ),
    locationGetter.meta_data.iloc[0].to_dict(),
    pv_model="pysamv1",
    config_files={
        "pv": "/home/tford/dev/InSPIRE/Studies/USMap_Doubleday_2024/SAM/01/01_pvsamv1.json"
    },
)

# for i in range(25):
#     for j in range(25):
#         if i == j:
#             print("same year")
#             continue

#         if res["poa_rear"][j * 8760 : (j * 8760) + 1] == res["poa_rear"][i * 8760 : i * 8760 + 1]:
#             print("equal")
#         else:
#             print(i, j, "not equal")

In [ ]:
inspire_ds = pvdeg.pysam.inspire_ground_irradiance(
    locationGetter.weather_data.isel(
        gid=0
    ).to_dataframe(),  # function handles UTC -> local for us
    locationGetter.meta_data.iloc[0].to_dict(),
    config_files={
        "pv": "/home/tford/dev/InSPIRE/Studies/USMap_Doubleday_2024/SAM/01/01_pvsamv1.json"
    },
)

In [ ]:
inspire_ds

In [ ]:
geo_weather, geo_meta = locationGetter.geospatial_data()

geo_meta = geo_meta[geo_meta["state"] != "Alaska"]
geo_weather = (
    geo_weather.sel(gid=geo_meta.index).chunk({"gid": 4}).isel(gid=slice(0, 1600))
)  # exclusive upper bound

In [ ]:
shapes = {"annual_poa": ("gid",), "ground_irradiance": ("gid", "time", "distance")}

template = pvdeg.geospatial.output_template(
    ds_gids=geo_weather,  # times will cause error, fix below
    shapes=shapes,
    add_dims={
        "distance": 10
    },  # this will autogenerate a range of length 10 for the coordinate axis
)

# modified range as produced by the corrected times for the tmy dataset
template["time"] = pd.date_range(start="2001-01-01 00:30:00", freq="1h", periods=8760)

In [ ]:
client.scatter(geo_meta, broadcast=True)
# we do not want to scatter weather because it is larger than memory and will cause problems

In [ ]:
# 401 groups
# we have dont this with a few of them before
geo_weather

In [ ]:
geo_weather.isel(gid=slice(4, 10))

In [ ]:
# 25 tasks at a time but this is the same as applying in blocks of 4

pysam_tiny_res = pvdeg.geospatial.analysis(
    weather_ds=geo_weather,
    meta_df=geo_meta,
    func=pvdeg.pysam.inspire_ground_irradiance,
    template=template,
    config_files={
        "pv": "/home/tford/dev/InSPIRE/Studies/USMap_Doubleday_2024/SAM/01/01_pvsamv1.json"
    },
)

In [ ]:
pysam_tiny_res.to_netcdf(
    "downsample9-conf01-pysam-ground-irradiance.nc", engine="h5netcdf"
)

In [ ]:
import os

print(os.getcwd())

In [ ]:
pysam_tiny_res

In [ ]:
res = pvdeg.pysam.inspire_ground_irradiance(
    geo_weather.isel(gid=0).to_dataframe(),
    geo_meta.iloc[0].to_dict(),
    config_files={
        "pv": "/home/tford/dev/InSPIRE/Studies/USMap_Doubleday_2024/SAM/01/01_pvsamv1.json"
    },
)

In [ ]:
res.ground_irradiance[0][1:]

In [ ]:
loaded = xr.open_dataset(
    "downsample20-conf01-pysam-ground-irradiance.nc", engine="h5netcdf"
).compute()